## Observations and Insights 

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from scipy.stats import linregress

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
mouse_data_merged = mouse_metadata.merge(study_results, left_on="Mouse ID", right_on="Mouse ID", how="inner")

# Display the data table for preview
mouse_data_merged.head()


In [ ]:
# Checking the number of mice.
meta_count = mouse_metadata["Mouse ID"].nunique()
study_count = study_results["Mouse ID"].nunique()
merged_count = mouse_data_merged["Mouse ID"].nunique()
print(f'Mouse counts: meta: {meta_count}, study: {study_count}, merged {merged_count}')

In [ ]:
# From instructions: "Before beginning the analysis, check the data for any mouse ID with duplicate time points and remove 
# any data associated with that mouse ID"

# I am reading this to mean that any mouse ID with duplicated time points is suspect/invalid and all data associated with
# mouse ID must be removed from the file

# Looking at data for mouse with duplicated timepoints reinforces this interpretation because column values vary by
# timepoint, indicating that Mouse ID was entered incorrectly (duplicated another mouse), or there was something wrong with
# the way that the data was collected/entered for this ID with duplicated timepoints

In [ ]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
# Create a group by on ID and Timepoint - any Mouse ID/timepoint with counts greater than 1 indicates duplicates
mouse_group = mouse_data_merged.groupby(["Mouse ID", "Timepoint"])
df = mouse_group.count() 
df = df.loc[df["Drug Regimen"]>1, :]

# Create a list of mouse IDs with duplicate timepoints
mouse_group2 = df.groupby(["Mouse ID"])
mice_with_duplicated_timepoints = mouse_group2.count()
mice_with_duplicated_timepoints = mice_with_duplicated_timepoints.reset_index()
mice_with_duplicated_timepoints

In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 

bad_mice_data = mice_with_duplicated_timepoints.merge(mouse_data_merged, left_on="Mouse ID", right_on="Mouse ID", 
                                                      how="left")
rows_to_drop = len(bad_mice_data)

bad_mice_data

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
# Outer join mice with duplicates to the orginal merged dataframe
# records with no match on mice with duplicates will have NaN values in the resulting fields from mice with duplicates
clean_mice_data = mice_with_duplicated_timepoints.merge(mouse_data_merged, left_on="Mouse ID", right_on="Mouse ID", 
                                                        how="outer", suffixes=("_bad",""))

# count rows before dropping mice with duplicates
rows_before_drop = len(clean_mice_data)

# identify rows to keep: rows that had no match in the outer join to mice_with_duplicated_timepoints
# drop rows that have non-zero values after replacing NaN with zero
#
# OBS!  This works in this instance, but not a good practice to continue.  At least not without additional
# data cleaning to ensure that all the values in Sex are valid (i.e. there were no zeroes in this column before
# replacing all the NaN values with zero)
#
clean_mice_data = clean_mice_data.fillna(0)
clean_mice_data = clean_mice_data.drop(clean_mice_data[clean_mice_data.Sex_bad >0].index)

# remove duplicated columns with non-values
clean_mice_data = clean_mice_data.drop([col for col in clean_mice_data if col.endswith('_bad')], 1)

# count rows in cleaned dataframe
rows_after_drop = len(clean_mice_data)

clean_mice_data.head()

In [ ]:
# Checking the number of mice in the clean DataFrame.

# print row counts for validation
print(f'Row count before drop: {rows_before_drop}')
print(f'Count of rows to drop: {rows_to_drop}')
print(f'Row count after drop (clean dataframe): {rows_after_drop}')


## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.

# create a groupby on "Drug Regimen" and then calculate statistics, appending to new df as each list is calced
regimen_group = clean_mice_data.groupby(["Drug Regimen"])
regimen_summary_df = regimen_group["Tumor Volume (mm3)"].mean().reset_index()\
        .rename(columns={"Tumor Volume (mm3)":"Mean"})
regimen_summary_df = pd.merge(regimen_summary_df, regimen_group["Tumor Volume (mm3)"].median(), on="Drug Regimen")\
        .rename(columns={"Tumor Volume (mm3)":"Median"})
regimen_summary_df = pd.merge(regimen_summary_df, regimen_group["Tumor Volume (mm3)"].var(), on="Drug Regimen")\
        .rename(columns={"Tumor Volume (mm3)":"Variance"})
regimen_summary_df = pd.merge(regimen_summary_df, regimen_group["Tumor Volume (mm3)"].std(), on="Drug Regimen")\
        .rename(columns={"Tumor Volume (mm3)":"Std Dev"})
regimen_summary_df = pd.merge(regimen_summary_df, regimen_group["Tumor Volume (mm3)"].sem(), on="Drug Regimen")\
        .rename(columns={"Tumor Volume (mm3)":"SEM"})

regimen_summary_df


In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line

regimen_summary2_df = regimen_group.agg({"Tumor Volume (mm3)" : ["mean", "median", "var", "std", "sem"]})

regimen_summary2_df

## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pandas.

# get count of mice on each regimen from the regimen groupby created above
count_df = regimen_group["Tumor Volume (mm3)"].count().reset_index()\
        .rename(columns={"Tumor Volume (mm3)":"Count of Mice"})

# create bar plot using pandas
count_df.plot.bar(x="Drug Regimen", y="Count of Mice", rot=45,legend=False,title="Count of Mice by Drug Regimen")
plt.show()


In [ ]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pyplot.

# get data for x and y axes
y_axis = count_df["Count of Mice"].tolist()
x_labels = count_df["Drug Regimen"].tolist()
x_axis = np.arange(len(y_axis))

# create bar chart using matplotlib 
fig = plt.figure()
plt.bar(x_axis, y_axis, color="#1f77b4", alpha=1, align="center",width=0.5)
fig.suptitle("Count of Mice by Drug Regimen", fontsize=12)
plt.xticks(x_axis, x_labels, rotation="45")
plt.xlabel('Drug Regimen')
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas
# To do: figure out how to get rid of "Count" on the left side of the pie chart

# copy clean mice data to another df in order to rename "Mouse ID" to "Count"
# To do: figure out how to get rid of label on the left side of the pie chart
# in the meantime, "Count" makes more sense as a label than "Mouse ID"
df = clean_mice_data.rename(columns={"Mouse ID":"Count"})

# count number of unique "Mouse ID" ("Count") by gender ("Sex")
gender_df = df.groupby("Sex")["Count"].nunique()

gender_df.plot.pie(title="Distribution of Mice by Gender")
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot

# reset index on gender data frame in order to pull gender labels and counts into separate lists
gender_df2 = gender_df.reset_index()
gender_list = gender_df2["Sex"].tolist()
gender_count = gender_df2["Count"].tolist()

# plot pie chart with matplotlib
fig = plt.figure()
plt.pie(gender_count, labels=gender_list)
fig.suptitle("Distribution of Mice by Gender", fontsize=12)
plt.show()

## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest


## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin

# get records for Capomulin regimen
Capomulin_records_df = clean_mice_data.loc[(clean_mice_data["Drug Regimen"] == "Capomulin"), :]

# get records for one mouse on Capomulin
One_mouse_records_df = Capomulin_records_df.loc[(clean_mice_data["Mouse ID"] == Capomulin_records_df.iloc[0]["Mouse ID"]), :]
mouse_shown = One_mouse_records_df.iloc[0][0]

# get list tumor volumes for the mouse
tumor_volumes_list = One_mouse_records_df["Tumor Volume (mm3)"].tolist()

# get list of time points
timepoints_list = One_mouse_records_df["Timepoint"].tolist()

# format title for plot
title = 'Tumor volume of mouse ' + mouse_shown + ' on Capomulin'

# create line plot
fig = plt.figure()
plt.plot(timepoints_list, tumor_volumes_list)
fig.suptitle(title, fontsize=12)
plt.xlabel('Timepoints')
plt.ylabel('Tumor Volumes')
plt.show()


In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen
weight_group = clean_mice_data.groupby(["Weight (g)"])
weight_summary_df = weight_group.agg({"Tumor Volume (mm3)" : "mean"})

weight_summary_df = weight_summary_df.reset_index()
weight_list2 = weight_summary_df["Weight (g)"].tolist()
avg_tumor_vol_list = weight_summary_df["Tumor Volume (mm3)"].tolist()

fig = plt.figure()
plt.scatter(weight_list2, avg_tumor_vol_list, marker="o", facecolors="red")
fig.suptitle("Avg tumor volume vs. weight (Capomulin)", fontsize=12)
plt.xlabel('Weight (g)')
plt.ylabel('Avg Tumor Volume')
plt.show()



## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen

# run linregress to get components for correlaton coefficient and regression model
(slope, intercept, rvalue, pvalue, stderr) = linregress(weight_list2, avg_tumor_vol_list)

# calc correlation coefficient and format regression model equation
correlation_coefficient = round(rvalue**2, 2)
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

print ()
print (f'Correlation coefficient: {correlation_coefficient}')
print (f'Linear regression model: {line_eq}')
print ()
